In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
#go to the website
try:
    driver = webdriver.Chrome()
    url = 'https://getsquire.com/discover/united-kingdom'
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located('h3')) #ensure all h3 tags are loaded
except Exception as e:
    print(type(e).__name__, str(e))
    
time.sleep(10)

InvalidArgumentException Message: invalid argument: invalid locator
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00007FF7BF39AD02+56930]
	(No symbol) [0x00007FF7BF30F602]
	(No symbol) [0x00007FF7BF1C42E5]
	(No symbol) [0x00007FF7BF2097BC]
	(No symbol) [0x00007FF7BF209A2C]
	(No symbol) [0x00007FF7BF24A967]
	(No symbol) [0x00007FF7BF22BCDF]
	(No symbol) [0x00007FF7BF2481E2]
	(No symbol) [0x00007FF7BF22BA43]
	(No symbol) [0x00007FF7BF1FD438]
	(No symbol) [0x00007FF7BF1FE4D1]
	GetHandleVerifier [0x00007FF7BF716F8D+3711213]
	GetHandleVerifier [0x00007FF7BF7704CD+4077101]
	GetHandleVerifier [0x00007FF7BF76865F+4044735]
	GetHandleVerifier [0x00007FF7BF439736+706710]
	(No symbol) [0x00007FF7BF31B8DF]
	(No symbol) [0x00007FF7BF316AC4]
	(No symbol) [0x00007FF7BF316C1C]
	(No symbol) [0x00007FF7BF3068D4]
	BaseThreadInitThunk [0x00007FFA45D77374+20]
	RtlUserThreadStart [0x00007FFA47A7CC91+33]



In [3]:
#Get provinces
provinces = driver.find_elements(By.TAG_NAME, 'h3')#each province
province = [i.text for i in provinces]
province #list of provinces

['England', 'Northern Ireland', 'Scotland', 'Wales']

In [4]:
#find the cities for each province
cities = {}
for i in range(1, len(province)+1):
    city = driver.find_elements(By.XPATH, f'//*[@id="__next"]/div/div[2]/div[2]/div[{i}]/div')
    city = [name.find_elements(By.TAG_NAME, 'a') for name in city]
    cities[province[i-1]] = [name.text for name in city[0]]
cities

{'England': ['Alcester',
  'Alderley Edge',
  'Alton',
  'Basingstoke',
  'Bedford',
  'Benfleet',
  'Billericay',
  'Bilston',
  'Bingley',
  'Birmingham',
  'Bolton',
  'Boston',
  'Bourne End',
  'Bournemouth',
  'Brandon',
  'Bridgnorth',
  'Brighton',
  'Bristol',
  'Broadstone',
  'Brockenhurst',
  'Bromley',
  'Burton-on-Trent',
  'Bury St. Edmunds',
  'Camberley',
  'Chelmsford',
  'Cheltenham',
  'Coventry',
  'Croydon',
  'Daventry',
  'Devizes',
  'Doncaster',
  'Dronfield',
  'East Grinstead',
  'Eastbourne',
  'Epsom',
  'Esher',
  'Evesham',
  'Farnham',
  'Fleet',
  'Fordingbridge',
  'Gateshead',
  'Gloucester',
  'Greenford',
  'Guildford',
  'Halifax',
  'Harrogate',
  'Haslemere',
  'Henley-in-Arden',
  'Hornchurch',
  'Hove',
  'Ilford',
  'Ingatestone',
  'Kenilworth',
  'Kingston upon Thames',
  'Leamington Spa',
  'Leeds',
  'Leicester',
  'Leigh',
  'Liverpool',
  'London',
  'Loughton',
  'Luton',
  'Lymington',
  'Maidstone',
  'Maldon',
  'Manchester',
  'Mil

In [5]:
#create function for city url
def city_url (province, city):
    province = province.lower()
    province = province.replace(' ', '-')
    city = city.lower()
    city = city.replace(' ', '-')
    return f'https://getsquire.com/discover/united-kingdom/{province}/{city}'

In [6]:
#get the city urls in a list
city_urls =[]
for keys, values in cities.items():
    for y in values:
        city_urls.append(city_url(keys,y))

In [7]:
city_urls

['https://getsquire.com/discover/united-kingdom/england/alcester',
 'https://getsquire.com/discover/united-kingdom/england/alderley-edge',
 'https://getsquire.com/discover/united-kingdom/england/alton',
 'https://getsquire.com/discover/united-kingdom/england/basingstoke',
 'https://getsquire.com/discover/united-kingdom/england/bedford',
 'https://getsquire.com/discover/united-kingdom/england/benfleet',
 'https://getsquire.com/discover/united-kingdom/england/billericay',
 'https://getsquire.com/discover/united-kingdom/england/bilston',
 'https://getsquire.com/discover/united-kingdom/england/bingley',
 'https://getsquire.com/discover/united-kingdom/england/birmingham',
 'https://getsquire.com/discover/united-kingdom/england/bolton',
 'https://getsquire.com/discover/united-kingdom/england/boston',
 'https://getsquire.com/discover/united-kingdom/england/bourne-end',
 'https://getsquire.com/discover/united-kingdom/england/bournemouth',
 'https://getsquire.com/discover/united-kingdom/england

In [8]:
#get cards (where info on the barber shops are kept)
def get_url(url):
    try:
        driver.get(url)
        wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "[data-automationid='shopcard']")))
        cards = driver.find_elements(By.CSS_SELECTOR, "[data-automationid='shopcard']")
        print(f"found {len(cards)} cards")
        return cards
    except Exception as error:
        print("unable to reach card due to:", error)
    

In [9]:
#create dictionary to store Barber shop data
data = {
        "Name":[],
        "Address":[],
        "City":[],
        "Province":[],
        "Link":[]
        }

In [10]:
#split the card content into more data columns
def parser (cards, url):
    for card in cards:
        name = card.find_element(By.TAG_NAME,'h2').text
        data['Name'].append(name)
        address = card.find_element(By.TAG_NAME,'p').text
        data['Address'].append(address)
        data['Link'].append(card.find_element(By.TAG_NAME,'a').get_attribute('href'))
        data['City'].append(url.split('/')[-1])
        data['Province'].append(url.split('/')[-2])

In [11]:
#fill in the data columns
for url in city_urls:
    try:
        cards = get_url(url)
        parser(cards, url)
    except Exception as card_error:
        print(f"can't process due to{card_error}")

found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 2 cards
found 2 cards
found 1 cards
found 1 cards
found 1 cards
found 6 cards
found 1 cards
found 1 cards
found 1 cards
found 2 cards
found 1 cards
found 1 cards
found 3 cards
found 5 cards
found 2 cards
found 1 cards
found 3 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 2 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 2 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 2 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 2 cards
found 7 cards
found 1 cards
found 4 cards
found 21 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found 4 cards
found 2 cards
found 1 cards
found 1 cards
found 1 cards
found 1 cards
found

In [12]:
#convert to dataframe
df = pd.DataFrame(data)

In [13]:
df

,Name,Address,City,Province,Link
0,Scallywags Barbers,"6 Kinwarton Rd, Alcester, ENG, B49 6QT",alcester,england,https://getsquire.com/discover/barbershop/scal...
1,FORA,"69a London Road, Alderley Edge, ENG, SK9 7DY",alderley-edge,england,https://getsquire.com/discover/barbershop/fora...
2,Headcase Barbers - Alton,"3c Normandy Street, Alton, ENG, GU34 1DD",alton,england,https://getsquire.com/discover/barbershop/head...
3,Taylor Made Barbershop,"15a London Street, Basingstoke, ENG, RG21 7NT",basingstoke,england,https://getsquire.com/discover/barbershop/tayl...
4,McNally Barbers,"73 Bower Street, Bedford, ENG, MK40 3RB",bedford,england,https://getsquire.com/discover/barbershop/mcna...
...,...,...,...,...,...
233,Downtown Barbers - Stenny,"1A Tryst Road, Larbert, SCT, FK5 4QQ",larbert,scotland,https://getsquire.com/discover/barbershop/down...
234,A.L BARBERS,"Space2B @ the Maltings, First Floor, house 1.,...",dinas-powys,wales,https://getsquire.com/discover/barbershop/a-l-...
235,Freestyle Barbers,"64A Portfield, Haverfordwest, WLS, SA61 1BW",haverfordwest,wales,https://getsquire.com/discover/barbershop/free...
236,Rogues & Gentlemen barbers,"41 High street, Menai Bridge, WLS, LL59 5EF",menai-bridge,wales,https://getsquire.com/discover/barbershop/rogu...


In [14]:
#write dataframe to csv file
df.to_csv('Barber.csv')